<a href="https://colab.research.google.com/github/kobemawu/www/blob/master/LDA_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
path_label = [["business",0],["entertainment",1],["food",2],["graphics",3],["historical",4],["medical",5],["politics",6],["space",7],["sport",8],["technologie",9]]

In [20]:
import os

# データの読み込み
text_list = []
label_list = []

def dir_check(path,label):
    files = os.listdir('archive/'+path)
    for file in files:
        file_check('archive/'+path+'/'+file,label)
        
def file_check(path,label):
    f = open(path, 'r', encoding='UTF-8')
    
    text = f.read()
    text_list.append(text)
    label_list.append(label)
    
    f.close()

for pl in path_label:
    dir_check(pl[0],pl[1])

In [21]:
# トークン化
from nltk.tokenize import word_tokenize

token_list = []

for text in text_list:
    tokens = word_tokenize(text)
    token_list.append(tokens)

In [22]:
docs = token_list

## データロード・前処理

## 前処理 

In [56]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<","$","%","@","]","[","|","!",'*',"’","”","“","="]                  \
         +["say","one","two","three","us","also","mr","would","first","could","new","use","make","get","take","may","many"] \
         +en_stop

In [57]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport
import re

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    pattern1 = '.*\d'
    res = re.match(pattern1, word)
    if res:
        return None
    
    pattern2 = '.*\''
    res = re.match(pattern2, word)
    if res:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

<>:19: DeprecationWarning: invalid escape sequence '\d'
<>:19: DeprecationWarning: invalid escape sequence '\d'
/tmp/ipykernel_13/3784310338.py:19: DeprecationWarning: invalid escape sequence '\d'
  pattern1 = '.*\d'


## LDA準備

In [58]:
import gensim
from gensim import corpora

In [59]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

## LDA学習

In [99]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=5,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [100]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=3)
for topic in topics:
    print(topic)

(0, '0.004*"design" + 0.003*"people" + 0.002*"food"')
(1, '0.006*"war" + 0.003*"world" + 0.003*"german"')
(2, '0.003*"people" + 0.003*"film" + 0.002*"war"')
(3, '0.003*"war" + 0.003*"german" + 0.003*"government"')
(4, '0.003*"year" + 0.003*"people" + 0.003*"world"')


In [101]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[0:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.3903055), (4, 0.60542685)]
document ID 1:[(4, 0.9968282)]
document ID 2:[(2, 0.241932), (3, 0.62987095), (4, 0.12614194)]
document ID 3:[(0, 0.9987281)]
document ID 4:[(1, 0.40293562), (3, 0.24911329), (4, 0.3466789)]
document ID 5:[(0, 0.41492876), (4, 0.5830723)]
document ID 6:[(4, 0.9970829)]
document ID 7:[(1, 0.40077344), (3, 0.5968855)]
document ID 8:[(4, 0.9975303)]
document ID 9:[(2, 0.9966693)]


In [102]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [108]:
n=1

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
#print(" ".join(docs[n]))

[(4, 0.99682826)]
['grain']


## 可視化

In [104]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [105]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [106]:
#上で可視化したモデルをgoogle drive上にsave

pyLDAvis.save_html(lda_display,'vis5.html')

In [26]:
!ls

sample_data  vis.html
